In [82]:
dimension = 3
perimeter = 25
sensitivity = 1
jsonfilename = "matrices4x4ver01.json"

In [104]:
def main() -> None:
    output_data = []
    for iteration in range(100):
        B = randomMatrix(dimension, perimeter)
        data = abnormality_check(B)
        if data != None:
            output_data.append(abnormality_check(B))
    print_the_output(output_data)
    into_json(format_data(output_data))


def abnormality_check(B) -> dict:
    B_red = B.LLL()
    v_min = shortestVector(B_red)[1]
    x = B.solve_left(v_min) # find linear combination corresponding to shortest vector found by LLL
    G = gram_matrix(B)
    for i in range(G.nrows()):
        data = innerComparsion(dimension, perimeter, B, B_red, v_min, x, G,i) # terrible
        if data != None:
            return data
main()

print("Done")

<class 'sage.modules.free_module_element.FreeModuleElement_generic_dense'>
<class 'sage.modules.free_module_element.FreeModuleElement_generic_dense'>
[ -8   0  -3]
[-12  23 -20]
[ 21  -7  12] :  B
[  73  156 -204]
[ 156 1073 -653]
[-204 -653  634] :  G
(6, 1, 3) :  The linear combination given by LLL
(7.3528, 1.0000, 3.3958) :  The linear combination suggested by cube
(-1.3528, 0.00000, -0.39584) :  The difference
(3, 2, -2) :  Shortest vector given by LLL
(0.49060, -0.77090, -1.3082) :  Shortest vector given by cube
[-14.64   -0.0  -1.61]
[   0.0    0.0    0.0]
[ 11.25    0.0   1.88] :  Diff impact analysis

Done


In [106]:
mat = matrix([[ -1 ,  9,   1],
           [ -1 ,  6 , -4],
           [  2, -10,   8]])

dictionary = abnormality_check(mat)
for pair in dictionary.items():
    print(pair[1], ": ", pair[0])


[ -1   9   1]
[ -1   6  -4]
[  2 -10   8] :  B
[ 83  51 -84]
[ 51  53 -94]
[-84 -94 168] :  G
(1, -8, -4) :  The linear combination given by LLL
(1.0000, -9.8824, -5.0294) :  The linear combination suggested by cube
(0.00000, 1.8824, 1.0294) :  The difference
(-1, 1, 1) :  Shortest vector given by LLL
(-1.1765, 0.00000, 0.29413) :  Shortest vector given by cube
[  -0.0    0.0    0.0]
[  -0.0  21.26  -7.75]
[   0.0 -19.38   8.48] :  Diff impact analysis


### batshit cool math

In [102]:
def innerComparsion(dimension, perimeter, B, B_red, v_min, x, G,i) -> dict:
    solutions = find_real_minimum(G, i, x[i])
    for j in range(dimension - 1):  # zbytecny forcyklus???
        difference = x[j] - solutions[j]
        if abs(difference) > sensitivity: 
            if all([sol == 0 for sol in solutions]):# FIME fuj
                break  
            return into_dict(B, G, x, solutions)
        
def find_real_minimum(G, i, x) -> vector:
    matrixA = matrix(G.nrows() - 1, G.nrows() - 1, 0)
    matrixB = matrix(G.nrows() - 1, 1, 0)
    matrixA[0,0] = 1
    a, b = 0, 0
    for row in range(G.nrows()):
        if row != i:
            matrixA[a] = [G[row, j] for j in range(len(G[row])) if j != i]
            matrixB[b] = sum([x * G[row,j] for j in range(len(G[row])) if j == i])
            a += 1
            b += 1
    # insert indices
    result = (matrixA.solve_right((-1)*matrixB)).list()
    result.insert(i,x)
    return vector(result).n(digits=5) 


def multiply_B_by_diff(B, diff) -> matrix:
    B = matrix_to_list(B)
    n = len(B)
    diff = vector_to_list(diff)
    for i in range(n):
        for j in range(n):
            B[i][j] = round(B[i][j]*  diff[i] * diff[j], 2)
    return matrix(B)

### output formatting

In [10]:
import json

def format_data(output_data):
    for dic in output_data:
        for key, value in dic.items():
            if isinstance(value, sage.matrix.matrix_integer_dense.Matrix_integer_dense):
                dic[key] = matrix_to_list(value)
            else:
            # FIXME here I assume that output_data consits only of matrices and one-line entries (vectors, free modules)
                dic[key] = vector_to_list(value)
    return output_data

def into_json(output_data):
    out_file = open(jsonfilename, "w+") 
    json.dump(output_data, out_file) 
    out_file.close() 


def print_the_output(output_data) -> None:
    for dict in output_data:
        for pair in dict.items():
            print(pair[1], ": ", pair[0])
        print()

### uninteresting tools

In [105]:
from random import randint, seed


def shortestVector(matrix): 
    """
    Returns the norm of the shortest vector and its index.
    """
    return sorted([(matrix[idx].norm().n(), matrix[idx], idx) for idx in range(matrix.nrows())])[0]

def randomMatrix(dimension, per) -> matrix:
    list = [randint(-per, per) for _ in range(dimension**2)]
    M = matrix(ZZ, dimension, dimension, list)
    while M.rank() != dimension:
        list = [randint(-per, per) for _ in range(dimension**2)]
        M = matrix(ZZ, dimension, dimension, list)
    return M

def gram_matrix(matrix) -> matrix:
    """
    Calculate Gram matrix of given matrix.
    :param matrix: given matrix
    :return: Gram matrix of matrix
    """
    return matrix * matrix.transpose()

def matrix_to_list(A) -> list:
    return [[int(A.list()[row * A.ncols() + col]) for col in range(A.ncols())] for row in range(A.nrows())]

def vector_to_list(vector) -> list:
    return [float(num) for num in vector.list()]


def into_dict(B, G, x, solutions) -> dict:
    result = {}
    result["B"] = B
    result["G"] = G
    result["The linear combination given by LLL"] = x
    result["The linear combination suggested by cube"] = solutions
    result["Shortest vector given by LLL"] = x*B
    result["Shortest vector given by cube"] = vector(solutions)*B
    result["The difference of lincombs"] = x - vector(solutions)
    result["Diff impact analysis"] = multiply_B_by_diff(B, x - vector(solutions))
#     result["LLL.norm"] = vector(x*B).norm().n(digits=5)
#     result["cube.norm"] = (vector(solutions)*B).norm().n(digits=5)
    return result